In [1]:
import pymanopt
import numpy as np

dim=3

In [421]:
def random_cov():
    A = np.random.rand(dim-1, dim-1)
    return np.dot(A, A.transpose())

In [2]:
man = pymanopt.manifolds.Sphere(dim)
A = np.random.rand(dim-1, dim-1)
cov = np.dot(A, A.transpose())
mean = man.random_point()

In [3]:
mean, cov

(array([-0.44328158, -0.46141816,  0.76850161]),
 array([[1.26839351, 0.32717228],
        [0.32717228, 0.13089927]]))

In [299]:
# generate tangent space
N = 100
samples = np.random.multivariate_normal(np.zeros(dim-1), cov, N)

In [300]:
# We need a tangent
normal = mean/np.linalg.norm(mean)

In [301]:
tol = 1e-5

def gram_schmidt(vectors):
    Q = np.zeros_like(vectors)
    for i in range(len(vectors)):
        v = vectors[i]
        for j in range(i):
            proj = np.dot(v, Q[j]) / np.dot(Q[j], Q[j]) * Q[j]
            v = v - proj
        Q[i] = v / np.linalg.norm(v)
    return Q
    
def get_axis(normal):
    # Assumes it is norm 1
    Q = np.eye(dim)
    arg = np.argmax(normal)
    if normal[arg]>(1-tol) and np.sum(normal)>(1-tol):
        return np.concatenate([Q[:arg], Q[arg+1:]])
    return gram_schmidt(np.vstack([normal,Q[:-1]]))[1:]

def get_e(i,n):
    return np.array(i*[0]+[1]+(n-i-1)*[0])

In [302]:
ax = get_axis(normal)

# Project points onto the axis to get all points in the tangent space
tangent_points = samples@ax

points = man.exp(mean, tangent_points)

In [431]:
# For estimateing normalisation constant
def estimate_normalising(mu, Sigma, manifold, m=None, S=100):
    d = manifold.dim
    z = np.sqrt((2*np.pi)**d*np.linalg.det(Sigma))
    samples = np.random.multivariate_normal(np.zeros(d), Sigma, S)
    vs = samples@ax
    return  z*np.mean(compute_vol(mu, vs, manifold, m))

sphere_M = lambda v: np.array([np.eye(v.shape[-1])]*len(v))

def compute_vol(mu, v, manifold, m=None):
    if m == None: m = sphere_M
    return np.sqrt(np.linalg.det(m(manifold.exp(mu, v-mu))))

In [432]:
def extrinsic_to_log(manifold, mu, x, ax):
    point = manifold.log(mu, x)
    return np.dot(point, ax.T)

In [457]:
def objective_grad_mu(points, mu, Sigma, ax, manifold, S=100):
    d = manifold.dim
    samples = np.random.multivariate_normal(np.zeros(d), Sigma, S)
    vs = samples@ax
    ms = compute_vol(mu, vs, manifold)
    z = np.sqrt((2*np.pi)**d*np.linalg.det(Sigma))
    grad = -(np.array([extrinsic_to_log(manifold,mu,p,ax) for p in points])
             .mean(0)-z*(ms.reshape(1,-1)@samples)/
             (S*estimate_normalising(mu, Sigma, manifold)))
    return grad

def objective_grad_A(points, mu, Sigma, ax, manifold, S=100):
    d = manifold.dim
    vals, vecs = np.linalg.eig(Sigma)
    A = (vecs@np.diag(1/np.sqrt(vals))).T
    samples = np.random.multivariate_normal(np.zeros(d), Sigma, S)
    ms = compute_vol(mu, vs, manifold)
    term2 = np.zeros((d, d)).astype(dtype='float64')
    for m,s in zip(ms,samples):
        term2 += m*((s.reshape(-1,1))@(s.reshape(1,-1)))
    term2 *= np.sqrt((2*np.pi)**d*np.linalg.det(Sigma))
    term2 /= (S*estimate_normalising(mu, Sigma, manifold))
    term1 = np.zeros((d, d)).astype(dtype='float64')
    for p,v in zip(points,vs):
        log = extrinsic_to_log(manifold,mu,p,ax)
        term1 += (log.reshape(-1,1))@(log.reshape(1,-1))
    term1 /= len(points)
    return A@(term1-term2)

def objective(points, mu, Sigma, manifold):
    d = manifold.dim
    result = 0
    inv = np.linalg.inv(Sigma)
    for p in points:
        log = extrinsic_to_log(manifold,mu,p,ax)
        result += np.dot(log, inv@log)
    result /= 2*len(points)
    return result + np.log(estimate_normalising(mu, Sigma, manifold))

def convergence_criteria(points, manifold, e=1e-4):
    x =  lambda mu0, Sigma0, mu, Sigma: (objective(points, mu, Sigma, manifold)-
                                         objective(points, mu0, Sigma0, manifold))
    return (lambda mu0, Sigma0, mu, Sigma: np.abs(x(mu, Sigma, mu0, Sigma))>e)
def mle_manifold(points, manifold, step_size_mu=1e-2, step_size_A=1e-2):
    d = manifold.dim
    Sigma0 = random_cov()
    mu0 = manifold.random_point()
    Sigma = random_cov()
    mu = manifold.random_point()
    criterion = convergence_criteria(points, manifold)
    count, max_loops = 0, 100
    while criterion(mu0, Sigma0, mu, Sigma) and count < max_loops:
        norm_const = estimate_normalising(mu, Sigma, manifold)
        grad_mu = objective_grad_mu(points, mu, Sigma, ax, manifold)@ax
        mu0, Sigma0 = mu, Sigma
        mu = manifold.exp(mu0, step_size_mu*grad_mu)[0]
        norm_const = estimate_normalising(mu, Sigma0, manifold)
        vals, vecs = np.linalg.eig(Sigma)
        A = (vecs@np.diag(1/np.sqrt(vals))).T
        grad_A = objective_grad_A(points, mu, Sigma0, ax, manifold)
        A -= step_size_A*grad_A
        Sigma = np.linalg.inv(A.T@A)
        count += 1
    return mu, Sigma
    
    

In [458]:
mle_manifold(points, man)

55


(array([ 0.06631592,  0.84624475, -0.29386839]),
 array([[0.47957189, 0.14276516],
        [0.14276516, 0.48519716]]))

In [460]:
mean

array([-0.44328158, -0.46141816,  0.76850161])

In [462]:

cov

array([[1.26839351, 0.32717228],
       [0.32717228, 0.13089927]])